## Introduction

Cross products (or Cartesian joins) are an important concept in SQL for joining tables together. However, they are seldom used as-is for reasons we will investigate in this exercise.

The files we will use in this exercise are pre-processed from data taken from the Government of Alberta's Open Data Portal and are licensed by the [Open Government License](https://open.alberta.ca/licence).

In [3]:
import pandas as pd

oil_records = pd.read_csv("OilProductionByMunicipality_2018.csv")
oil_records.head()

gas_records = pd.read_csv("NaturalGasProductionByMunicipality_2018.csv")
gas_records.head()

well_records = pd.read_csv("WellCountByMunicipality_2018.csv")
well_records.head()


,CSDUID,CSD,Period,IndicatorSummaryDescription,UnitOfMeasure,OriginalValue
0,4805026,Drumheller,2018,Well Count,NaN,5.0
1,4805031,Starland County,2018,Well Count,NaN,17.0
2,4805041,Kneehill County,2018,Well Count,NaN,36.0
3,4802001,Warner County No. 5,2018,Well Count,NaN,11.0
4,4807049,Wainwright No. 61,2018,Well Count,NaN,32.0


Recall that a cross product between tables A $(A_1, A_2, ...,A_n)$ and B $(B_1, B_2, ..., B_m)$ will take the form

A $\times$ B $(A_1 B_1, A_1 B_2, ..., A_1 B_m, A_2 B_1, ..., A_n B_m)$

Given this, give a naive implementation of the cross product of `oil_records` and `well_records`, using the [`append()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html) method in pandas.

In [4]:
resultTuples = []

for oil_tuple in oil_records.itertuples():
    for well_tuple in well_records.itertuples():
        resultTuples.append(oil_tuple[1:] + well_tuple[1:])
     
resultFrame = pd.DataFrame(resultTuples)

resultFrame.set_axis(oil_records.columns.append(well_records.columns), axis = 1, inplace = True)
resultFrame.describe()

,CSDUID,Period,OriginalValue,CSDUID,Period,UnitOfMeasure,OriginalValue
count,6.068000e+03,6068.0,6.068000e+03,6.068000e+03,6068.0,0.0,6068.000000
mean,4.810379e+06,2018.0,1.607874e+06,4.810565e+06,2018.0,NaN,64.219512
std,5.263328e+03,0.0,5.854682e+06,5.181746e+03,0.0,NaN,131.286085
min,4.801003e+06,2018.0,5.640000e+01,4.801003e+06,2018.0,NaN,0.000000
25%,4.806014e+06,2018.0,7.737060e+04,4.806016e+06,2018.0,NaN,1.000000
50%,4.811006e+06,2018.0,1.964535e+05,4.811033e+06,2018.0,NaN,13.500000
75%,4.814003e+06,2018.0,8.972984e+05,4.815013e+06,2018.0,NaN,41.000000
max,4.819071e+06,2018.0,4.743823e+07,4.819071e+06,2018.0,NaN,671.000000


pandas was created to provide more efficient ways to handle tabular data. While no Cartesian product has been [officially implemented yet](https://github.com/pandas-dev/pandas/issues/5401), a frequently mentioned workaround works as follows: 

- create a dummy column on both A and B (The column should just hold a single value for all rows of A and B)
- use `merge()` to join both tables on the dummy column
- drop the dummy column from the resulting column

Try this now on `gas_records` and `well_records`. 


In [5]:
cartesianFrame = gas_records.assign(dummy = 1).merge(well_records.assign(dummy = 1), on = 'dummy').drop('dummy', 1)

cartesianFrame.describe()

,CSDUID_x,Period_x,OriginalValue_x,CSDUID_y,Period_y,UnitOfMeasure_y,OriginalValue_y
count,6.232000e+03,6232.0,6.232000e+03,6.232000e+03,6232.0,0.0,6232.000000
mean,4.810343e+06,2018.0,1.760190e+06,4.810565e+06,2018.0,NaN,64.219512
std,5.288762e+03,0.0,5.228492e+06,5.181734e+03,0.0,NaN,131.285801
min,4.801003e+06,2018.0,5.790000e+01,4.801003e+06,2018.0,NaN,0.000000
25%,4.806011e+06,2018.0,9.116687e+04,4.806016e+06,2018.0,NaN,1.000000
50%,4.811006e+06,2018.0,3.198679e+05,4.811033e+06,2018.0,NaN,13.500000
75%,4.814256e+06,2018.0,1.002641e+06,4.815013e+06,2018.0,NaN,41.000000
max,4.819071e+06,2018.0,3.568238e+07,4.819071e+06,2018.0,NaN,671.000000


## merge() and join() methods

Now use the pandas `merge()` and `join()` methods to accomplish the following tasks:

Use merge() to combine the gas_records and well_records on the CSDUID columns. What is the cardinality of the resulting table?

Use join() to combine the oil_records and well_reocrds on the CSD column. What is the cardinality of the resulting table?

What would be appropriate tables (and columns) to use a left or right SQL-style join upon? What would be the intended result?

In [11]:
gas_well = pd.merge(oil_records, well_records, on = 'CSDUID')
display(gas_well.head())

oil_well = oil_records.join(well_records, )

,CSDUID,CSD_x,Period_x,IndicatorSummaryDescription_x,UnitOfMeasure_x,OriginalValue_x,CSD_y,Period_y,IndicatorSummaryDescription_y,UnitOfMeasure_y,OriginalValue_y
0,4805026,Drumheller,2018,Oil Production,m3,6969.6,Drumheller,2018,Well Count,NaN,5.0
1,4805031,Starland County,2018,Oil Production,m3,112882.5,Starland County,2018,Well Count,NaN,17.0
2,4805041,Kneehill County,2018,Oil Production,m3,280762.7,Kneehill County,2018,Well Count,NaN,36.0
3,4802001,Warner County No. 5,2018,Oil Production,m3,195193.9,Warner County No. 5,2018,Well Count,NaN,11.0
4,4807049,Wainwright No. 61,2018,Oil Production,m3,1086087.1,Wainwright No. 61,2018,Well Count,NaN,32.0


In [ ]:
# code using join() here